# Launching LCModel

## Prepare environment

In [ ]:
!pip install vip-client


In [ ]:
from vip_client import VipSession
from pathlib import *
from os.path import exists
from os import listdir, getcwd
import IPython

# disable local session backup
VipSession._BACKUP_LOCATION = None

## Set up VIP execution

### First, copy paste you VIP API key from the VIP portal (in "My Account")

In [ ]:
VipSession.init(api_key="TOCHANGE");

In [ ]:
# Instantiate a Session with a name
VipSession._BACKUP_LOCATION = None
my_session = VipSession("lcmodel")

### Specify where are your input data

In [ ]:
# upload the input data, by default it is a "dataset" directory but you can change that
dataset = Path("dataset")

if not listdir(dataset):
    raise RuntimeError(f"[{dataset}] directory must not be empty") 
my_session.upload_inputs(dataset)

In [ ]:
# take one of the dataset signal file as LCModel input in the next step
! tree {dataset}

### Select the input file you want to use

In [ ]:
# change the file name to one of your signal
signal_file_name = "TOCHANGE"

signal_file = dataset / signal_file_name
if not exists( signal_file):
    raise RuntimeError(f"[{signal_file}] signal file does not exist") 

# define the other setting from vip hosted files
input_settings = { 
    "signal_file" : signal_file,
    "zipped_folder": "/vip/Home/LCModel/basis.zip",
    "makebasis_file" : "/vip/Home/LCModel/makeBasis_3T_VIP.in",
    "control_file" : "/vip/Home/LCModel/fit_3T_lcmodel02_vip.control"
}

## The execution is ready to be launched
### You will have to wait a few minutes for the execution to be over

In [ ]:
# launch the execution and wait for it to finish
my_session.launch_pipeline(pipeline_id = "LCModel/0.2", input_settings = input_settings)
my_session.monitor_workflows()

### Now you are ready to download the results

In [ ]:
# download the output and clean VIP data
my_session.download_outputs()
my_session.finish()

# Now you can try to view the pdf result

First execute the next cell and verify the expected path

In [ ]:
# This block get the pdf result local path from the vip client info
execution_outputs = next(iter(my_session.workflows.values()))["outputs"]
#find pdf result
pdf_result = next( output for output in execution_outputs if output["path"].endswith(".pdf"))
# get local path
pdf_relative_path = pdf_result["path"][len(my_session.vip_output_dir)+1:]
pdf_path = Path(my_session.local_output_dir) / pdf_relative_path
print(pdf_path)

# This should open a pdf viewer in any notebook environment

In [ ]:
IPython.display.IFrame(pdf_path, width=600, height=600)

## This method should only work in JupyterLab and fail in other python environments

Please change the prefix to the relative path of your notebook inside your jupyterlab root directory.

Then execute the cell and click on the button below. It should open a jupyterlab tab with the pdf.

In [ ]:
prefix = "lcmodel/"

from IPython.display import Markdown as md
md("<button data-commandLinker-command=\"docmanager:open\" data-commandLinker-args='{\"path\":\"" + prefix + str(pdf_path) + "\"}'  href=\"#\">Open pdf</button>")